# Problem Description
Click [here](https://code.aliyun.com/mindopt001/mindopt-open-examples/blob/master/doc/example_1.md?spm=5176.21102998.J_6338734050.1.637f5e8dfU1WxM&file=example_1.md).

# About
- Enviroment: Anaconda, python 3.8.10 64-bit, Windows 10.
- Using main algorithm: Taboo search.

## Solution Form
The best form for solutions is a numpy matrix. Considering the possible modification in the future, I wrapped it with `class solution`.

In [ ]:
import numpy as np

class solution(np.ndarray):
    pass

# Initialization
sl = solution(np.arange(1, 10, 1))


## Adjacent Solution Generator
As the lands and plants both have the upper bound, it's nessesary to adjust the solutions to make them valid.
We will generate 4 solutions each time, Each solution was from the base solution after change one row's values randomly.
So, I generated a row of random values with -1, 0 and 1 in every iteration, and accumulated it in the row.
```cpp
    def adjacentSolution(self, x:solution)->tuple:
        adjSolutionSet = []
        rowChange = np.array(x[0, :])

        for i in range(self.rows):

            for j in range(self.cols):
                rowChange[j] = rd.randint(-1, 1)

            tmp = np.array(x)
            tmp[i] += rowChange
            adjSolutionSet.append(tmp)

        # Check whether these solutions are valid.

        return tuple(adjSolutionSet)
```

# class TS
Main algorithm, including tabu search and visualization.
## Tabu list
A `list` object, its also available to substitute it with `set` object to make it more efficient. But I will rewrite this application if I hope it run faster.

But the length of tabu list, so called *tabu length*, whose value may affect the convergence speed deeply. For this reason, I will set a control group and change the tabu length to reveal the relationships between them.

## Amnesty Rule
Here, if the tabu list is full, the first come will get out first. This rule make the tabu list like a *quene*. But till now, it still is a `list` object.

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import random as rd

class TS:
    """Get solution through here."""

    def __init__(self, lands:tuple, plants:tuple, production:np.ndarray, tabuLength:int) -> None:
        self.plants = plants
        self.lands = lands
        self.rows = len(plants)
        self.cols = len(lands)
        self.production = production
        self.optSolution = solution((self.rows, self.cols))

        self.tabuLength = tabuLength
        self.tabuList = list()

        # Visualize
        self.axesX = []
        self.axesY = []


    def isValid(self, x:solution)->bool:
        UsingLands=[0 for item in self.lands]
        PlantingPlants=[0 for item in self.plants]

        for i in range(self.rows):
            for j in range(self.cols):
                UsingLands[j] += x[i,j]
                PlantingPlants[i] += x[i,j]
                if x[i,j] < 0:
                    return False

        for i in range(self.cols):
            if UsingLands[i] > self.lands[i]:
                return False

        for i in range(self.rows):
            if PlantingPlants[i] > self.plants[i]:
                return False

        return True
        

    def adjacentSolution(self, x:solution)->list:
        adjSolutionSet = []
        rowChange = np.array(x[0, :])

        i = 0
        while i < self.rows:
            # Generate a vector of random numbers.
            for j in range(self.cols):
                rowChange[j] = rd.randint(-1, 1)

            tmp = np.array(x)
            tmp[i] += rowChange

            # Check whether it is valid and filter the invalid solutions.
            if self.isValid(tmp):
                adjSolutionSet.append(tmp)
                i += 1

        return adjSolutionSet


    def grade(self, x:solution)->int:
        sum = 0
        for i in range(self.rows):
            for j in range(self.cols):
                sum += x[i,j] * self.production[i, j]

        return sum


    def getOptimalSolution(self, origin: solution, iter_times:int)->solution:
        slt = np.array(origin)
        for k in range(iter_times):
            # print(len(self.tabuList), end=' ')
            adj = self.adjacentSolution(slt)
            adj.sort(key=lambda a:self.grade(a), reverse=True)

            # Choose the best one which is not in tabu list.
            for item in adj:
                if not isIn(item, self.tabuList):
                    # The optimal solution of each iteration. 
                    slt = item
                    self.optSolution = slt

                    self.axesX.append( k + 1 )
                    self.axesY.append(self.grade(slt))

                    while len(self.tabuList) >= self.tabuLength:
                        self.tabuList.pop(0)
                        
                    self.tabuList.append(item)

                    break

        # Find the best in tabu list.
        bst = np.zeros(slt.shape)
        for i in self.tabuList:
            # print(self.grade(i))
            if self.grade(bst) < self.grade(i):
                bst = i
        return bst

    def showChart(self):
        pl.plot(self.axesX, self.axesY, color='orange')
        pl.title("The Production-Iteration Chart")
        pl.xlabel("Iteration")
        pl.ylabel("Production")
        pl.show()

# Visualization
As usual, I choose **matplotlib** library as the visualizer.

In [ ]:
import numpy as np
import search as sc
import matplotlib.pyplot as plt

prod = np.array([[500,550,630,1000,800,700],
                 [800,700,600,950,90,930],
                 [1200,1040,980,860,880,780],
                 [1000,960,840,650,600,700]])
lands = (42,56,44,39,60,59)
plants = (76,88,40,96)
lenList = (10,60,110,160,210,260)
iterations = 2000 # 2k might be the optimal value of iterations.

ini = np.ones((4,6))
opt = np.zeros((4,6))
ag = None

fig, ax=plt.subplots()

print('Calculating...')
for l in lenList:
    ag = sc.TS(lands, plants, prod, l)
    ag.getOptimalSolution(ini, iterations)
    if ag.grade(opt) < ag.grade(ag.optSolution):
        opt = ag.optSolution

    ax.plot(ag.axesX, ag.axesY, label='Tabu list length = '+str(l))

ax.set_title('Production-Iteration Chart')
ax.set_xlabel('Iteration')
ax.set_ylabel('Production')
ax.legend()
ax.grid(axis='both')

print('Calculation done!')

print('The optimal solution is: ')
for i in range(opt.shape[0]):
    for j in range(opt.shape[1]):
        print('%8.0f'%opt[i,j], end=' ')
    else:
        print()
print('The production is {pd}'.format(pd=ag.grade(opt)))

plt.show()